In [1]:
import pandas as pd
from llama_index.core import Document
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext
from llama_index.core.response.notebook_utils import display_source_node

In [2]:
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [3]:
from llama_index.core.storage.docstore import SimpleDocumentStore

In [4]:
import pandas as pd
import re
import ast

In [5]:
data = pd.read_csv("results/advert_comparison_cleaned.csv")

In [22]:
features = [
    "Recruiting young people who are still in school",
    "Paying more than the market rate for the skill level or type of job that they are hiring for",
    "Not mentioning any skill requirements",
    "Not mentioning the nature of the job",
    "Not mentioning the name or the location of the hiring business",
    "Paying the same salary for different job posts positions",
    "Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media",
    "Recruiting specifically females for a job that male or female applicants would qualify for",
    "Unprofessional writing poor grammar spelling",
    "Recruiting models",
    "Changing from English to other languages in the middle of the post",
    "Using a suspicious email address",
    "Advertising for positions in several promises especially without detail",
    "Looks Legit",
]

In [8]:
data

,IDn,Advert,Monitor Rating,Monitor Reason,Recruiting young people who are still in school,Paying more than the market rate for the skill level or type of job that they are hiring for,Not mentioning any skill requirements,Not mentioning the nature of the job,Not mentioning the name or the location of the hiring business,Paying the same salary for different job posts positions,Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media,Recruiting specifically females for a job that male or female applicants would qualify for,Unprofessional writing poor grammar spelling,Recruiting models,Changing from English to other languages in the middle of the post,Using a suspicious email address,Advertising for positions in several promises especially without detail,Looks Legit
0,572495,Females 90 to work Cleaning . if you have powe...,7,Typical of the previous cases,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,572469,"ESKOM Is Taking CVs For General Worker,Office ...",9,Previous eskom mentioned that they do not adve...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,573359,Netcare Free Nursing 2024 Now Open At All 9 P...,9,The modus operandi of how the post is been wri...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,573172,Am looking for anyone who's really looking for...,8,"Worth of checking, the manner is been written ...",NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,572592,We are looking for a Nanny.Must be good in Eng...,7,"The post needs a little bit of research, howev...",NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
5,573343,I am looking for someone who wants to work on ...,7,"Does not look legit, and there is element of H...",NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,573283,For everyone who looking for a jobs Around Sou...,6,"LIttle bit of research will assist, at this st...",NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,573586,Lose 5kg in 1 week . Lose 8kg in 2 weeks. Lose...,2,MIght be a general scam than HT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
8,570625,Submit your CV today hurryServest Cleaning Job...,9,The manner the post is been written it send se...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
9,572631,Hi Ladies and GentsIf you are interested in mo...,7,"Does not look legit, and there is element of H...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [7]:
data_columns

['IDn',
 'Advert',
 'Monitor Rating',
 'Monitor Reason',
 'Recruiting young people who are still in school',
 'Paying more than the market rate for the skill level or type of job that they are hiring for',
 'Not mentioning any skill requirements',
 'Not mentioning the nature of the job',
 'Not mentioning the name or the location of the hiring business',
 'Paying the same salary for different job posts positions',
 'Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media',
 'Recruiting specifically females for a job that male or female applicants would qualify for',
 'Unprofessional writing poor grammar spelling',
 'Recruiting models',
 'Changing from English to other languages in the middle of the post',
 'Using a suspicious email address',
 'Advertising for positions in several promises especially without detail',
 'Looks Legit']

In [6]:
data_columns = list(data.columns)

In [24]:

new_cols = [re.sub("\W+", " ", col).strip() for col in data_columns]
rename = {}
for new_col, col in zip(new_cols, data_columns):
    rename[col] = new_col
data.rename(columns=rename, inplace=True)
list(data)

['IDn',
 'Advert',
 'Monitor Rating',
 'Monitor Reason',
 'Recruiting young people who are still in school',
 'Paying more than the market rate for the skill level or type of job that they are hiring for',
 'Not mentioning any skill requirements',
 'Not mentioning the nature of the job',
 'Not mentioning the name or the location of the hiring business',
 'Paying the same salary for different job posts positions',
 'Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media',
 'Recruiting specifically females for a job that male or female applicants would qualify for',
 'Unprofessional writing poor grammar spelling',
 'Recruiting models',
 'Changing from English to other languages in the middle of the post',
 'Using a suspicious email address',
 'Advertising for positions in several promises especially without detail',
 'Looks Legit']

In [25]:
rename = {'Unnamed 0':'IDn', 'Unnamed 1':'Advert'}

In [26]:
data.rename(columns=rename, inplace=True)

In [27]:
data.to_csv("results/advert_comparison_cleaned.csv", index=False)

In [28]:
stories = []
metadata = []
flags = {}
for idx, feature in enumerate(features):
    flags[feature] = f"```Finding: " + feature + "```"


In [33]:
for idx, row in data.iterrows():
    identified_features = []
    for feature in features:
        if row[feature] == 1:
            identified_features.append(flags[feature])
        else:
            data.loc[idx, feature] = None

    story = f"""The advert with text ```{row['Advert']}``` is given a Monitor Rating of {row['Monitor Rating']} where 0 is not suspicious and 9 is likely fraudulent or fake.  
    and the Monitor Reason for giving this rating is ```{row['Monitor Reason']}```.  The following 
    features were observed: {' AND '.join(identified_features)}. """
    stories.append(story)
    meta = {
        "idn": row["IDn"],
    }
    metadata.append(meta)


In [34]:
stories

['The advert with text ```Females 90 to work Cleaning . if you have power drop ur number down here we will call u. no certificate wanted.Monthly salary R7,500APPLY NOW//>>https://sites.google.com/view/za-applications/homeNo scam Thank u!!!!1.Limpopo2. North West3. Gauteng4. Mpumalanga5. Free State6. Northern Cape7. Eastern Cape8. Western Cape9. KwaZulu-NaalSubmit your CV:https://sites.google.com/view/sa-applications-/home``` is given a Monitor Rating of 7 where 0 is not suspicious and 9 is likely fraudulent or fake.  \n    and the Monitor Reason for giving this rating is Typical of the previous cases.  The following \n    features were observed: ```Finding: Not mentioning the name or the location of the hiring business``` AND ```Finding: Recruiting specifically females for a job that male or female applicants would qualify for``` AND ```Finding: Unprofessional writing poor grammar spelling```. ',
 'The advert with text ```ESKOM Is Taking CVs For General Worker,Office Cleaners, and hole

In [15]:

story = (
    "A monitor is a person who assesses a online recruitment adverts and provides a Likert-style rating as to the likelihood "
    "of this advert being used for the purposes of falsely luring respondents into trafficking.  The Likert scale "
    "ranges from 0 to 9, where 0 is not suspicious and 9 is likely fraudulent or fake.  The rating is known as the Monitor rating. "
    "The Monitor also notes a reason for their rating. "
    f"Here is a list of key findings a Monitor looks for in the advert: {features} "
    "When the Monitor has NO reason to suspect that the adverts is being used for the purposes of human trafficking, "
    "the finding 'Looks Legit' and ONLY 'Looks Legit' is chosen.  Therefore the Monitor HAS to choose AT LEAST one of these options. ",
)
stories.append(story)
documents = []
for meta, document in zip(metadata, stories):
    documents.append(Document(text=document, metadata=meta))


In [16]:
print(story)

("A monitor is a person who assesses a online recruitment adverts and provides a Likert-style rating as to the likelihood of this advert being used for the purposes of falsely luring respondents into trafficking.  The Likert scale ranges from 0 to 9, where 0 is not suspicious and 9 is likely fraudulent or fake.  The rating is known as the Monitor rating. The Monitor also notes a reason for their rating. Here is a list of key findings a Monitor looks for in the advert: ['Recruiting young people who are still in school', 'Paying more than the market rate for the skill level or type of job that they are hiring for', 'Not mentioning any skill requirements', 'Not mentioning the nature of the job', 'Not mentioning the name or the location of the hiring business', 'Paying the same salary for different job posts positions', 'Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media', 'Recruiting specifically females for a job that male

In [72]:
node_parser = HierarchicalNodeParser.from_defaults()

nodes = node_parser.get_nodes_from_documents(documents)
len(nodes)


leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)
root_nodes = get_root_nodes(nodes)
len(root_nodes)
# define storage context

docstore = SimpleDocumentStore()


In [73]:
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

llm = OpenAI(model="gpt-4o")
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)


In [74]:
def create_prompt(advert):
    query_str = (
    f"Assistant please provide a Monitor rating and a Monitor reason for the following advert ```{advert}```"
    "without using your pretrained knowledge about human trafficking but only the documentation given to you."
    "Please provide your Monitor rating on an integer scale of 0 to 9, with 9 being the most likely to be used in human trafficking."
    "Also provide a reason for your Monitor rating.  This reason must also NOT be extracted from anywhere else but the documentation given to you."
    "You will also need to provide a list of ALL your findings that you have identified in the advert as per the documentation given to you."
    "Findings can ONLY be one OR more of the following, don't make up new ones and do not blend any together as you see fit:"
    f"{features}"
    "When the finding is ```Looks Legit```, no OTHER finding can be true."
    "Provide your response in JSON format and ensure it can be parsed correctly."
    "Here is an example:"
    '{"Monitor rating": integer, "Monitor reason": "reasoning", "finding": ["finding 1", "finding 2", etc]}',
)
    return " ".join(query_str)

In [78]:
create_prompt(advert)

'Assistant please provide a Monitor rating and a Monitor reason for the following advert ```Netcare Free Nursing 2024 Now Open  At All 9 ProvincesWe Are Looking For 320 Woman To Start TrainingRequirements:Grade 10-12Monthly Allowance:R8080If you\'re interestedPlease Inbox Us We will Reply you immediately with the interview Date And LocationLink::https://sites.google.com/view/general-cleaners2024/home```without using your pretrained knowledge about human trafficking but only the documentation given to you.Please provide your Monitor rating on an integer scale of 0 to 9, with 9 being the most likely to be used in human trafficking.Also provide a reason for your Monitor rating.  This reason must also NOT be extracted from anywhere else but the documentation given to you.You will also need to provide a list of ALL your findings that you have identified in the advert as per the documentation given to you.Findings can ONLY be one OR more of the following, don\'t make up new ones and do not b

In [ ]:
query_engine = RetrieverQueryEngine.from_args(retriever)
response = query_engine.query(prompt)

In [79]:

base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
advert = data.sample(1)["Advert"].values[0]
# data.sample(1)["Unnamed 0"].values[0]
prompt = create_prompt(advert)
nodes = retriever.retrieve(prompt)
base_nodes = base_retriever.retrieve(prompt)


In [80]:

prompt

'Assistant please provide a Monitor rating and a Monitor reason for the following advert ```There is a job opening pharmacy opening looking for a Cashier who have experience in the pharmacy.Cashier must have worked in a pharmacy beforesuremedspharmacy13@gmail.comVanderbijlpark```without using your pretrained knowledge about human trafficking but only the documentation given to you.Please provide your Monitor rating on an integer scale of 0 to 9, with 9 being the most likely to be used in human trafficking.Also provide a reason for your Monitor rating.  This reason must also NOT be extracted from anywhere else but the documentation given to you.You will also need to provide a list of ALL your findings that you have identified in the advert as per the documentation given to you.Findings can ONLY be one OR more of the following, don\'t make up new ones and do not blend any together as you see fit:[\'Recruiting young people who are still in school\', \'Paying more than the market rate for 

In [81]:

for node in nodes:
    p = display_source_node(node, source_length=10000)

for node in base_nodes:
    display_source_node(node, source_length=10000)


query_engine = RetrieverQueryEngine.from_args(retriever)
base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

response = query_engine.query(prompt)
print(response.response)
print(advert,str(response))

**Node ID:** 495ed944-fe30-4487-a716-9017f30a81d1<br>**Similarity:** 0.8982243143443959<br>**Text:** The advert with text ```There is a job opening pharmacy opening looking for a Cashier who have experience in the pharmacy.Cashier must have worked in a pharmacy beforesuremedspharmacy13@gmail.comVanderbijlpark``` is given a Monitor Rating of 5 where 0 is not suspicious and 9 is likely fraudulent or fake.  
    and the Monitor Reason for giving this rating is The job post it has elements of HT or a job scam. The big red flag is a email address that is been provided.<br>

**Node ID:** 9726de12-35d6-4fb2-8650-76e6a471723f<br>**Similarity:** 0.8538416578103099<br>**Text:** and the Monitor Reason for giving this rating is LIttle bit of research will assist, at this stage it is 50/50 if it HT but it does look like there is element of possible deception..  The following 
    features were observed: ```Finding: Not mentioning any skill requirements``` AND ```Finding: Not mentioning the nature of the job``` AND ```Finding: Not mentioning the name or the location of the hiring business```.<br>

**Node ID:** 072e03e8-e830-4562-b28a-df5846c2582e<br>**Similarity:** 0.8534134915526826<br>**Text:** and the Monitor Reason for giving this rating is The post needs a little bit of research, however such post do turn to be not legitimate and it more of HT than job scam .  The following 
    features were observed: ```Finding: Not mentioning the name or the location of the hiring business``` AND ```Finding: Using a suspicious email address```.<br>

**Node ID:** b591c4be-55c4-400f-8b10-1f979161611b<br>**Similarity:** 0.8529963649975703<br>**Text:** and the Monitor Reason for giving this rating is The modus operandi of how the post is been written it fits well into the previous we have dealt with that turned into intercepts..  The following 
    features were observed: ```Finding: Not mentioning the name or the location of the hiring business``` AND ```Finding: Recruiting specifically females for a job that male or female applicants would qualify for```.<br>

**Node ID:** dbbdb449-20c5-4596-ae96-bc3ff5a8a4c3<br>**Similarity:** 0.8528258830976334<br>**Text:** and the Monitor Reason for giving this rating is Normal such job post does not require any experience or no enough information is not provided about the name of the alleged company and the location..  The following 
    features were observed: ```Finding: Not mentioning the name or the location of the hiring business```.<br>

...

**Node ID:** 072e03e8-e830-4562-b28a-df5846c2582e<br>**Similarity:** 0.8534134915526826<br>**Text:** and the Monitor Reason for giving this rating is The post needs a little bit of research, however such post do turn to be not legitimate and it more of HT than job scam .  The following 
    features were observed: ```Finding: Not mentioning the name or the location of the hiring business``` AND ```Finding: Using a suspicious email address```.<br>

**Node ID:** b591c4be-55c4-400f-8b10-1f979161611b<br>**Similarity:** 0.8529963649975703<br>**Text:** and the Monitor Reason for giving this rating is The modus operandi of how the post is been written it fits well into the previous we have dealt with that turned into intercepts..  The following 
    features were observed: ```Finding: Not mentioning the name or the location of the hiring business``` AND ```Finding: Recruiting specifically females for a job that male or female applicants would qualify for```.<br>

**Node ID:** dbbdb449-20c5-4596-ae96-bc3ff5a8a4c3<br>**Similarity:** 0.8528258830976334<br>**Text:** and the Monitor Reason for giving this rating is Normal such job post does not require any experience or no enough information is not provided about the name of the alleged company and the location..  The following 
    features were observed: ```Finding: Not mentioning the name or the location of the hiring business```.<br>

**Node ID:** 49dde63d-cb18-4cb8-8bad-d77dc0c27695<br>**Similarity:** 0.8501977495728724<br>**Text:** and the Monitor Reason for giving this rating is Typical of the previous cases.  The following 
    features were observed: ```Finding: Not mentioning the name or the location of the hiring business``` AND ```Finding: Recruiting specifically females for a job that male or female applicants would qualify for``` AND ```Finding: Unprofessional writing poor grammar spelling```.<br>

{
    "Monitor rating": 9,
    "Monitor reason": "The job post has elements of HT or a job scam. The big red flag is an email address that is provided.",
    "finding": ["Using a suspicious email address"]
}
There is a job opening pharmacy opening looking for a Cashier who have experience in the pharmacy.Cashier must have worked in a pharmacy beforesuremedspharmacy13@gmail.comVanderbijlpark {
    "Monitor rating": 9,
    "Monitor reason": "The job post has elements of HT or a job scam. The big red flag is an email address that is provided.",
    "finding": ["Using a suspicious email address"]
}


In [90]:
row

IDn                                                                                                                                                                572571
Advert                                                                                                                  Code 14 DriverLocation: MeyertonMust have vali...
Monitor Rating                                                                                                                                                          4
Monitor Reason                                                                                                          it looks legit, however it is worth doing reas...
Recruiting young people who are still in school                                                                                                                       NaN
Paying more than the market rate for the skill level or type of job that they are hiring for                                                          

In [94]:
row_entry

{'IDn': 572571,
 'Recruiting specifically females for a job that male or female applicants would qualify for': 1,
 'Not mentioning any skill requirements': 1,
 'Not mentioning the name or the location of the hiring business': 1,
 'Unprofessional writing poor grammar spelling': 1,
 "Hiring for an organization such as ESKOM who has publicly stated that they don't advertise job posts on social media": 1,
 'Recruiting young people who are still in school': 1,
 'Paying more than the market rate for the skill level or type of job that they are hiring for': 1,
 'Using a suspicious email address': 1,
 'Not mentioning the nature of the job': 1,
 'Recruiting models': 1,
 'Looks Legit': 1,
 'Changing from English to other languages in the middle of the post': 1}

In [95]:
response = query_engine.query(prompt)

findings = []
for idx, row in data.iterrows():
    row_entry = {}
    advert = row['Advert']
    row_entry['IDn'] = row['IDn']
    # print(idx)
    query_engine = RetrieverQueryEngine.from_args(retriever)
    prompt = create_prompt(advert)
    response = query_engine.query(prompt)
    for finding in ast.literal_eval(response.response)['finding']:
        row_entry[finding]=1
    row_entry['Monitor rating'\
       ] = ast.literal_eval(response.response)['Monitor rating']
    findings.append(row_entry)
    # print(response.response)
    print(idx, advert, str(response))

0 Females 90 to work Cleaning . if you have power drop ur number down here we will call u. no certificate wanted.Monthly salary R7,500APPLY NOW//>>https://sites.google.com/view/za-applications/homeNo scam Thank u!!!!1.Limpopo2. North West3. Gauteng4. Mpumalanga5. Free State6. Northern Cape7. Eastern Cape8. Western Cape9. KwaZulu-NaalSubmit your CV:https://sites.google.com/view/sa-applications-/home {
    "Monitor rating": 9,
    "Monitor reason": "The job post fits well into what other ST's normal post, so this definitely fits to HT, and possible evidence.",
    "finding": [
        "Recruiting young people who are still in school",
        "Not mentioning any skill requirements",
        "Not mentioning the nature of the job",
        "Not mentioning the name or the location of the hiring business",
        "Recruiting specifically females for a job that male or female applicants would qualify for",
        "Unprofessional writing poor grammar spelling",
        "Advertising for posit

In [101]:
data.columns

Index(['IDn', 'Advert', 'Monitor Rating', 'Monitor Reason',
       'Recruiting young people who are still in school',
       'Paying more than the market rate for the skill level or type of job that they are hiring for',
       'Not mentioning any skill requirements',
       'Not mentioning the nature of the job',
       'Not mentioning the name or the location of the hiring business',
       'Paying the same salary for different job posts positions',
       'Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media',
       'Recruiting specifically females for a job that male or female applicants would qualify for',
       'Unprofessional writing poor grammar spelling', 'Recruiting models',
       'Changing from English to other languages in the middle of the post',
       'Using a suspicious email address',
       'Advertising for positions in several promises especially without detail',
       'Looks Legit'],
      dtype='ob

In [102]:
cols = ['Recruiting young people who are still in school',
       'Paying more than the market rate for the skill level or type of job that they are hiring for',
       'Not mentioning any skill requirements',
       'Not mentioning the nature of the job',
       'Not mentioning the name or the location of the hiring business',
       'Paying the same salary for different job posts positions',
       'Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media',
       'Recruiting specifically females for a job that male or female applicants would qualify for',
       'Unprofessional writing poor grammar spelling', 'Recruiting models',
       'Changing from English to other languages in the middle of the post',
       'Using a suspicious email address',
       'Advertising for positions in several promises especially without detail',
       'Looks Legit']

In [104]:
df

,IDn,Recruiting young people who are still in school,Not mentioning any skill requirements,Not mentioning the nature of the job,Not mentioning the name or the location of the hiring business,Recruiting specifically females for a job that male or female applicants would qualify for,Unprofessional writing poor grammar spelling,Advertising for positions in several promises especially without detail,Hiring for an organization such as ESKOM who has publicly stated that they don't advertise job posts on social media,Paying more than the market rate for the skill level or type of job that they are hiring for,Using a suspicious email address,Recruiting models,Looks Legit,Changing from English to other languages in the middle of the post
0,572495,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,572469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,573359,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,573172,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,572592,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
5,573343,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,573283,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,573586,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,570625,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,572631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [103]:
df = pd.DataFrame(findings)
df = df[['IDn']+cols]

KeyError: "['Paying the same salary for different job posts positions', 'Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media'] not in index"

['IDn',
 'Recruiting young people who are still in school',
 'Paying more than the market rate for the skill level or type of job that they are hiring for',
 'Not mentioning any skill requirements',
 'Not mentioning the nature of the job',
 'Not mentioning the name or the location of the hiring business',
 'Paying the same salary for different job posts positions',
 'Hiring for an organization such as ESKOM who has publicly stated that they don t advertise job posts on social media',
 'Recruiting specifically females for a job that male or female applicants would qualify for',
 'Unprofessional writing poor grammar spelling',
 'Recruiting models',
 'Changing from English to other languages in the middle of the post',
 'Using a suspicious email address',
 'Advertising for positions in several promises especially without detail',
 'Looks Legit']

In [98]:
df.fillna(0)

,IDn,Recruiting young people who are still in school,Not mentioning any skill requirements,Not mentioning the nature of the job,Not mentioning the name or the location of the hiring business,Recruiting specifically females for a job that male or female applicants would qualify for,Unprofessional writing poor grammar spelling,Advertising for positions in several promises especially without detail,Hiring for an organization such as ESKOM who has publicly stated that they don't advertise job posts on social media,Paying more than the market rate for the skill level or type of job that they are hiring for,Using a suspicious email address,Recruiting models,Looks Legit,Changing from English to other languages in the middle of the post
0,572495,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,572469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,573359,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,573172,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,572592,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,573343,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,573283,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,573586,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,570625,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,572631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [107]:
response

ast.literal_eval(response.response)['Monitor rating']

4